In [1]:
# open gwas_clean_riskAllele_ids_with_beta8.txt and read each line and split it into a list
with open('snp_with_pvalue8.txt') as f:
    columns = []
    for line in f:
        # split the line into a list and append it to the columns list
        columns.append(line.strip().split())
# flatten the list of lists into a single list
columns = [item for sublist in columns for item in sublist]
# print the first 5 columns
print(columns[:5])

['rs143332484_T', 'rs616338_T', 'rs75932628_T', 'rs139967528_G', 'rs593742_A']


In [ ]:
import pandas as pd
data = pd.read_csv('../adni_full.raw', delim_whitespace=True, usecols=columns, nrows=5)

In [4]:
data = pd.read_csv('../adni_full.raw', delim_whitespace=True, usecols= lambda x: not x.endswith('.'), nrows=5)

/tmp/ipykernel_9263/2558506764.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv('../adni_full.raw', delim_whitespace=True, usecols= lambda x: not x.endswith('.'), nrows=5)


In [6]:
len(data.columns)

592538

In [7]:
for col in data.columns:
    if col in columns:
        print(col, "is in the list")

rs6882046_A is in the list
rs2718058_A is in the list
rs11771145_G is in the list
rs10498633_G is in the list
rs8106922_A is in the list
rs5167_T is in the list
